In [1]:
import pandas as pd
pd.set_option('display.max_rows', 100)
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter


## Imports

In [3]:
job = pd.read_csv('personality_jobs_done.csv')
colors = pd.read_csv('personality_colors.csv')
music = pd.read_csv('personality_music_genre.csv')

## CLeaning

In [4]:
job = job.drop(['firsPercentage','secondPercentage','thirdPercentage','fourthPercentage'],axis=1)

#remove (%) from personality_type
job.personality_type = job.personality_type.str[:4]
job.dropna(subset=['name','personality_type'], inplace=True)
for col in job.columns[2:]:
    job[col] = job[col].str.replace('%', '')
# remove XXXX class
job = job[~(job.personality_type == 'XXXX')]
# fill na with -1
job.fillna(-1, inplace=True)
#change object to int8
job[job.columns[2:]] = job[job.columns[2:]].astype('int8')

job.reset_index(inplace=True)
job = job.drop('index',axis=1)

In [5]:
colors = colors.drop(['web-scraper-order','web-scraper-start-url','links','links-href','pages',
                      'firsPercentage','secondPercentage','thirdPercentage','fourthPercentage',
                      'pages-href'],axis=1)

#remove (%) from personality_type
colors.personality_type = colors.personality_type.str[:4]
colors.dropna(subset=['name','personality_type'], inplace=True)
for col in colors.columns[2:]:
    colors[col] = colors[col].str.replace('%', '')
# remove XXXX class
colors = colors[~(colors.personality_type == 'XXXX')]
# fill na with -1
colors.fillna(-1, inplace=True)
#change object to int8
colors[colors.columns[2:]] = colors[colors.columns[2:]].astype('int8')

colors.reset_index(inplace=True)
colors = colors.drop('index',axis=1)

In [6]:
# clean unused columns
music.drop(['web-scraper-order', 'web-scraper-start-url', 'links', 'links-href', 'firsPercentage',
                 'secondPercentage', 'thirdPercentage', 'fourthPercentage', 'pages', 'pages-href'], axis=1, inplace=True)

music.dropna(subset=['name','personality_type'], inplace=True)
#remove (%) from personality_type
music.personality_type = music.personality_type.str[:4]
# remove % char from architype
for col in music.columns[2:]:
    music[col] = music[col].str.replace('%', '')
# remove XXXX class
music = music[~(music.personality_type == 'XXXX')]
# fill na with -1
music.fillna(-1, inplace=True)
#change object to int8
music[music.columns[2:]] = music[music.columns[2:]].astype('int8')

music.reset_index(inplace=True)
music = music.drop('index',axis=1)

## Project

In [11]:
class Job_recog():
   
    def __init__(self):
        self.job = pd.read_csv('clean_job.csv',index_col=0)
        self.music = pd.read_csv('clean_music.csv',index_col=0)
        self.colors = pd.read_csv('clean_colors.csv',index_col=0)
        
        self.color_similarity = cosine_similarity(self.colors[self.colors.columns[2:]])
        self.music_similarity = cosine_similarity(self.music[self.music.columns[2:]])
        self.job_similarity = cosine_similarity(self.job[self.job.columns[2:]])
        
        
    def find_similarity(self,title,data,q_type, use_col):
        colors_id = data[data[use_col] == title].index.values[0]       
        score = list(enumerate(q_type[colors_id]))
        
        sorted_score = sorted(score, key=lambda x:x[1], reverse= True)[1:]
        personality_list = []
        name_list = []
        for i,val in enumerate(sorted_score):
            name_list.append(data[data.index == val[0]]['name'].values[0])
            personality_list.append(data[data.index == val[0]]['personality_type'].values[0])
            if i == 2:
                break
        return name_list, personality_list
    
    def run(self,color, music):
        # First find color
        color_name, color_personality = self.find_similarity(color,self.colors,self.color_similarity,'name')
        print('Probably your persobality type is: ',Counter(color_personality).most_common(1)[0][0])
        print(f"Also, we recommend you this colors: {color_name}\n")

        # Second find music
        music_name, music_personality = self.find_similarity(music,self.music,self.music_similarity,'name')
        print('Probably your persobality type is: ',Counter(music_personality).most_common(1)[0][0])
        print(f"Also we recommend you this music genre: {music_name}\n")
        
        personalities = Counter(music_personality + color_personality).most_common(2)
        
        for personality in personalities:
            job_name, job_personality = self.find_similarity(personality[0],self.job,self.job_similarity,'personality_type')
            print(f'for this type of personality "{job_personality[0]}" our job recommendation: {job_name}')
        # job_name, job_personality = self.find_similarity(test1,self.job,self.job_similarity,'personality_type')
        return 0

## Results

In [12]:
test1 = Job_recog()
test1.run('Black','Electronicore')

Probably your persobality type is:  INTJ
Also, we recommend you this colors: ['Midnight Blue', 'Shadow', 'Dioxazine']

Probably your persobality type is:  INTP
Also we recommend you this music genre: ['Black MIDI', 'Electroacoustic music', 'Lowercase']

for this type of personality "INTP" our job recommendation: ['Audio Engineer', 'Geostationary Satellite Monitor', 'Hacker']
for this type of personality "INTJ" our job recommendation: ['Film Producer', 'Plague Doctor', 'Neuroscientist']


0